***Computer Vision project***

Import of library

In [ ]:
!pip install torchvision


In [ ]:
!pip install openai==0.28

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install gpt4all

In [ ]:
!pip install deepface

In [ ]:
!pip install ultralytics

In [ ]:
!pip install --upgrade transformers accelerate

In [ ]:
!pip install bitsandbytes

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import torch
import openai
import requests
import torch.nn as nn
from torchvision import models, transforms, datasets
from torchvision.datasets import FER2013
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_b0
from deepface import DeepFace
from ultralytics import YOLO
from sklearn.cluster import KMeans
from diffusers import StableDiffusionPipeline
from PIL import Image
from openai import OpenAI

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Upload the dataset Fer2013

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip -q fer2013.zip


Transform for the images

In [ ]:
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

Divided the dataset for the training and testing phase

In [ ]:
train_data = datasets.ImageFolder(root='train', transform=transform)
test_data = datasets.ImageFolder(root='test', transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, num_workers=4, pin_memory=True)

Load the model : Resnet18, Densenet121

In [ ]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 7)

In [ ]:
"""model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Replace classifier for FER2013 (7 emotions)
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, len(train_data.classes))

model = model.to(device)"""


In [ ]:
criterion= nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(val_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

Train the model to detect emotions

In [ ]:
model = model.to(device)
num_epochs= 20
for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()



        scheduler.step()
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(train_loader):.4f} - "
              f"Acc: {100*correct/total:.2f}%")

In [ ]:
!nvidia-smi

In [ ]:
#torch.save(model.state_dict(), 'modeldense.pth')
#model.load_state_dict(torch.load('model.pth'))

In [ ]:
model.load_state_dict(torch.load('Model 2.pth'))

Test the model

In [ ]:
def test(model, test_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy, all_preds, all_labels


In [ ]:
test_loss, test_acc, y_pred, y_true = test(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")


Extract caracteristics from the face on a image

In [ ]:
def predict_emotion(image_path, model=None, device="cpu"):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (48, 48))
        img = img.astype('float32') / 255.0
        img = torch.tensor(img).unsqueeze(0).unsqueeze(0)
        img = img.repeat(1, 3, 1, 1).to(device)

        if model is not None:
            model.eval()
            with torch.no_grad():
                output = model(img)
                _, predicted = torch.max(output, 1)
                return int(predicted.item())
        else:
            # fallback
            return "happy"
    except Exception as e:
        print(f"[WARN] Could not predict emotion: {e}")
        return "neutral"


In [ ]:
def extract_traits(image_path):
    traits = {}
    # Age + gender
    try:
        analysis = DeepFace.analyze(img_path=image_path, actions=["age", "gender"], enforce_detection=False)
        traits['age'] = analysis[0]['age']
        traits['gender'] = analysis[0]['gender']
    except:
        traits['age'], traits['gender'] = None, None

    # Glasses
    try:
        model_yolo = YOLO("yolov8n.pt")
        results = model_yolo(image_path)
        objects = [results[0].names[int(box.cls)] for box in results[0].boxes]
        traits['has_glasses'] = 'glasses' in objects or 'sunglasses' in objects
    except:
        traits['has_glasses'] = None

    # Dominant color
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (100, 100))
        img = img.reshape((-1, 3))
        kmeans = KMeans(n_clusters=1, n_init=10)
        kmeans.fit(img)
        b, g, r = kmeans.cluster_centers_[0]
        if r > 150 and g < 100 and b < 100:
            traits['top_color'] = "red"
        elif g > 150 and r < 100:
            traits['top_color'] = "green"
        elif b > 150 and r < 100:
            traits['top_color'] = "blue"
        else:
            traits['top_color'] = "unknown"
    except:
        traits['top_color'] = None

    if traits is None:
      return {}
    return traits



In [ ]:
generator = None

In [ ]:
def emotion_number_to_word (emotion) :
  emotion_map = {
      0:"angry",
      1:"disgusted",
      2:"fearful",
      3:"happy",
      4:"sad",
      5:"suprised",
      6:"neutral",

  }

  if isinstance(emotion, (int,float)):
    return emotion_map.get(int(emotion), "neutral")
  elif isinstance(emotion, str):
    return emotion.lower()
  else:
    return "neutral"

In [ ]:
def get_emo_strategy(emotion) :
  strategies = {
      "angry" :"trying to calm down and manage the feelimgs and to relativise " ,
      "disgusted" :"finding beauty and positivity in simple things" ,
      "fearful": " discovering courage, trying to surpass yourself " ,
      "happy" : " sharing this joy and try to keep it even more",
      "sad" : " finding the good side in the situation and have faith and remembering good memories",
      "suprised" :"embracing chances, enjoying life and embracing new experiences",
      "neutral" : " trying to be in positives vibes and to be proud of what we do today",

  }

  return strategies.get(emotion,"finding peace and confort")


In [ ]:
import random
from transformers import pipeline
import torch

In [ ]:
generator = pipeline(
                "text-generation",
                model="microsoft/phi-3-mini-4k-instruct",
                device=0 if torch.cuda.is_available() else -1,
                pad_token_id=50256
            )
print("✅ AI model ready!")


In [ ]:
ai_prompts = {
            'sad': [
                "A sad child discovers that every tear waters the seeds of hope, and soon",
                "When a little one feels down, a magical friend appears to show that",
                "A child learning that sadness is temporary finds out that",
                "In a moment of tears, a gentle voice whispers the truth that"
            ],
            'happy': [
                "A joyful child spreads happiness by sharing it, discovering that",
                "A happy little one learns that joy grows bigger when",
                "A child's laughter creates ripples of love that",
                "A cheerful child finds that happiness is most beautiful when"
            ],
            'fearful': [
                "A scared child finds incredible courage hiding inside, realizing that",
                "When fear tries to whisper lies, a brave child learns that",
                "A little one discovers that fear melts away when",
                "A child afraid of shadows finds that light always wins because"
            ],
            'angry': [
                "An angry child learns that feelings are visitors, not owners, and discovers that",
                "When frustration bubbles up, a wise child finds that peace comes when",
                "A child with big emotions learns that anger transforms into strength when",
                "A little one discovers that the heart heals fastest when"
            ],
            'surprised': [
                "A child amazed by life's wonders learns that magic is everywhere when",
                "An astonished little one discovers that surprises teach us that",
                "A child's wonder grows into wisdom as they learn that",
                "A curious child finds that the best surprises remind us that"
            ],
             'neutral': [
                "A calm child sits quietly and notices that peace grows inside when",
                "In a moment of stillness, a little one discovers that",
                "When everything feels just okay, a child realizes that even ordinary moments are",
                "A child resting in a gentle quiet learns that balance brings"
             ]
        }

In [ ]:
simple_stories = {
              'sad': [
                "Emma felt tears on her cheeks, but a gentle butterfly landed there and seemed to say, 'Your tears are pearls of courage.' Emma realized that crying shows how deeply she can feel love. She smiled, knowing her big heart was her superpower.",

                "Little Sam sat alone, feeling heavy inside. A wise old oak tree rustled its leaves and whispered, 'Even I bend in storms, but I never break.' Sam understood that being sad sometimes makes us stronger and more caring. He felt peaceful and proud.",

                "Maya's heart felt cloudy and gray. But then she noticed how rain makes flowers grow more beautiful. 'Sadness waters the garden of my soul,' she thought, feeling grateful for her ability to feel deeply and love fully.",

                "When Ben felt overwhelmed with sadness, his grandmother held his hand and said, 'The deepest rivers flow the quietest.' Ben learned that his sensitive heart was a gift that would help him understand and comfort others forever.",

                "Lily felt like she was carrying heavy stones in her chest. A singing bird showed her how to breathe deeply and let the sadness flow through her like a gentle stream. She felt lighter, knowing that all feelings are temporary visitors."
            ],

            'happy': [
                "Jake's joy was so bright that it lit up everyone around him. He learned that happiness shared becomes happiness doubled. His smile became a gift he gave to the world every day, making him feel even more wonderful inside.",

                "Sophie discovered that her laughter was like music that healed hearts. Every giggle created ripples of joy that reached people she'd never even met. She felt proud to be a bringer of light in the world.",

                "Max realized that his happiness wasn't just a feeling—it was a superpower that could turn ordinary moments into magical memories. He felt grateful for his ability to find joy in simple things and share it generously.",

                "When Chloe felt pure joy watching butterflies dance, she understood that happiness chooses those with open hearts. She felt blessed to be someone who could see beauty everywhere and spread sunshine wherever she went.",

                "Oliver's happiness bubbled up like a magical spring. He learned that joy isn't about having everything—it's about appreciating what's already wonderful. He felt rich with gratitude and contentment."
            ],

            'fearful': [
                "Anna felt scared, but then she remembered that brave doesn't mean 'not afraid'—it means 'afraid but doing it anyway.' She discovered that inside every fearful heart lives a mighty lion of courage, just waiting to roar. She felt incredibly brave and proud.",

                "Tim was trembling with fear until a gentle voice inside whispered, 'You've survived every scary moment so far.' He realized he was already a champion at being brave. His fear transformed into confidence and strength.",

                "Rose felt afraid to try something new, but then she imagined her future self cheering her on. 'You've got this!' her brave inner voice said. Rose felt supported by her own inner wisdom and stepped forward with courage.",

                "Leo discovered that fear was just excitement wearing a disguise. When he changed his thoughts from 'What if something goes wrong?' to 'What if something goes wonderfully?', his fear melted into anticipation and hope.",

                "Mia learned that fear is like a paper tiger—it looks scary but has no real power over her brave heart. She felt strong and capable, knowing that courage was her birthright and fear was just a temporary visitor."
            ],

            'angry': [
                "Sam felt anger burning in his chest, but then he imagined it as a dragon that needed gentleness, not fighting. He breathed kindness onto the anger-dragon until it transformed into a warm, protective feeling. Sam felt peaceful and in control.",

                "Grace was furious until she remembered that anger is just love with nowhere to go. She sent that love to herself instead, wrapping her heart in understanding. Grace felt calm and proud of her emotional wisdom.",

                "Alex discovered that underneath anger is always a need for love or understanding. When he gave himself the compassion he was seeking, the anger melted like snow in sunshine. Alex felt heard, valued, and at peace.",

                "Zoe felt anger rumbling inside like thunder, but then she realized thunder is just the sky releasing pressure so it can be clear again. She let her feelings flow through her gently, feeling refreshed and free afterward.",

                "Noah learned that anger is a teacher showing him what he cares about deeply. When he thanked his anger for trying to protect him, it softened into wisdom and strength. Noah felt grateful and empowered."
            ],

            'surprised': [
                "Ella's eyes went wide with wonder, and she realized that life is full of magical gifts waiting to be discovered. She felt grateful to have eyes that notice miracles and a heart open to amazement every single day.",
                "Oscar was amazed by an unexpected joy, learning that the universe loves to surprise those who pay attention. He felt special and loved, knowing that wonderful surprises were always around the corner for his curious heart.",
                "Ruby discovered something that took her breath away, realizing that wonder is the soul's way of celebrating life. She felt blessed to be someone who could still be amazed by the beauty and mystery all around her.",
                "Felix found an unexpected treasure and understood that the best surprises come to those with grateful hearts. He felt fortunate and loved by life itself, knowing more beautiful moments were always coming his way.",
                "Luna was delighted by a wonderful surprise that reminded her how much magic exists in ordinary days. She felt thankful for her ability to see wonder everywhere and looked forward to tomorrow's miracles."
            ],
              'neutral': [
                "Liam sat quietly and noticed how calm the world felt around him. He realized that being peaceful inside is just as magical as being joyful. He felt safe, steady, and ready for new adventures.",
                "Clara looked around on an ordinary day and noticed small wonders—the rustle of leaves, the warmth of sunlight, and the comfort of her own breath. She discovered that ordinary moments can be extraordinary treasures.",
                "Daniel felt neither sad nor excited, just still. Then he noticed how comforting balance can be. He learned that calmness is a gift that gives strength and clarity.",
                "Ava rested quietly, realizing that even when nothing special happens, her heart still shines gently. She felt content and grateful for the gift of simple peace."
            ]
        }

In [ ]:
def generate_story( emotion):
        if emotion in ai_prompts:
            prompt = random.choice(ai_prompts[emotion])

            print(prompt)
            try:
                result = generator(
                    "A child with big emotions learns that anger transforms into strength when",
                    max_length=200,
                    temperature=0.9,  # High creativity
                    do_sample=True,
                    num_return_sequences=1,
                    pad_token_id=50256
                )

                story = result[0]['generated_text'].replace(prompt, "").strip()
                # Clean up and limit length
                sentences = story.split('.')
                clean_story = '. '.join(sentences[:3]) + '.'

                if len(clean_story) > 50:
                    return clean_story
            except:
                pass
        #backup i case it fails
        stories = simple_stories.get(emotion,simple_stories[emotion])
        return random.choice(stories)

In [ ]:
def test_variety( emotion, num_tests=4):
        for i in range(num_tests):
            story = generate_story(emotion)
            print(f"\n Story {i+1}:")
            print(f"{story}")
            print("-"*40)


In [ ]:
test_variety("sad", 3)

In [ ]:
emotion_detected = "happy"
story = generate_story(emotion_detected)
print(f"Generated story: {story}")

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = build_structured_prompt(emotion, traits)

story = generator(prompt, max_length=300, temperature=0.8, do_sample=True)[0]["generated_text"]
print("📖 Story:\n", story)

In [ ]:
image_path = "/content/PrivateTest_134207.jpg"

emotion = predict_emotion(image_path, model, device)
story = generate_story(emotion_number_to_word(emotion))

print("Emotion:", emotion)
print("\n🧚 Story:\n", story)


In [ ]:
story ="he is a little boy, and he goes with his mother to the beach.  On the beach he is comforted by the gentle sounds of the ocean, and by a lovely woman, an olive-skinned woman, who is the captain of the _Warlord_ (a ship) and who serves the Captain-Captain of the _Warlord_, a ship that was launched on the day of the great earthquake as it is commonly reported, and which was launched for the purpose of defending the islands that it had attacked upon its arrival. The story of the _Warlord_ is not a long story."

In [ ]:
!pip install diffusers transformers accelerate safetensors

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("gsdf/Counterfeit-V2.5")
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

image = pipe(story, num_inference_steps=25, guidance_scale=7.5).images[0]

image.save("output10.png")

In [ ]:
prompt = "he is a little boy, and he goes with his mother to the beach.  On the beach he is comforted by the gentle sounds of the ocean, and by a lovely woman, an olive-skinned woman, who is the captain of the _Warlord_ (a ship) and who serves the Captain-Captain of the _Warlord_, a ship that was launched on the day of the great earthquake as it is commonly reported, and which was launched for the purpose of defending the islands that it had attacked upon its arrival. The story of the _Warlord_ is not a long story."
image = pipe(prompt, num_inference_steps=25, guidance_scale=7.5).images[0]

image.save("waifu_example4.png")

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "hakurei/waifu-diffusion"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

#prompt = "anime style, happy little girl with blue dress, sunny garden, magical atmosphere"
image = pipe(prompt, num_inference_steps=70, guidance_scale=7.5).images[0]

image.save("waifu_example2.png")

In [ ]:
image.save("output9.png")

In [ ]:
image.save("output6.png")

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("gsdf/Counterfeit-V2.5")
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")
negative_prompt = "nsfw, nude, naked, gore, blood, violence, disfigured, extra limbsmEasyNegative, extra fingers,fewer fingers"

image = pipe(story,negative_prompt, num_inference_steps=25, guidance_scale=7.5).images[0]

In [ ]:
image.save("output9.png")

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained("gsdf/Counterfeit-V2.5")
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

image = pipe(story, num_inference_steps=70, guidance_scale=7.5).images[0]

In [ ]:
image.save("output7.png")

In [ ]:
from transformers import pipeline
emotion_classifier = pipeline(
    "text-classification",
    model="SamLowe/roberta-base-go_emotions",
    return_all_scores=True
)

results = emotion_classifier(story)

best_emotion = max(results[0], key=lambda x: x['score'])

print(f"Émotion prédite: {best_emotion['label']} (score={best_emotion['score']:.2f})")


In [ ]:


emotion_image_classifier = pipeline(
    "image-classification",
    model="dima806/facial_emotions_image_detection"
)

image_path = "/content/output5.png"
image = Image.open(image_path)

results = emotion_image_classifier(image)

best_emotion = max(results, key=lambda x: x['score'])
print(f"Émotion détectée : {best_emotion['label']} (score={best_emotion['score']:.2f})")


In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


texts = ["neutral","happy","fearful"]
# Images
image_files = ["/content/output5.png", "/content/output6.png", "/content/output7.png","/content/output8.png"]
images = [Image.open(img) for img in image_files]

inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)

# Similarty
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)


for i, img_file in enumerate(image_files):
    print(f"\nImage: {img_file}")
    for text, prob in zip(texts, probs[i]):
        print(f" - Similarity with  '{text}': {prob:.4f}")


best_matches = torch.argmax(probs, dim=1)
for img_file, idx in zip(image_files, best_matches):
    print(f"\n✅ The image {img_file} correspond to the emotion: {texts[idx]}")
